In [3]:
from analysis import *
import dashboard

In [4]:
def check_residuals(data: pd.DataFrame,
                    time_col: str,
                    y_true: str,
                    y_predicted: str,
                    data_group: str,
                    period = 'D',
                    diff = 0):
    
    if period == 'D':
        num_lags = 40
    elif period == 'M':
        num_lags = 7
    else:
        return None

    temp = data.copy()
    temp[time_col] = pd.to_datetime(temp[time_col], format = '%Y-%m-%d')
    temp.index = pd.DatetimeIndex(temp.index)
    
    for cg in sorted(data[data_group].unique().tolist()):

        temp = temp[temp[data_group] == cg].resample(period).sum()

        temp['residual'] = temp[y_true] - temp[y_predicted]
        
        fig, ax = plt.subplots(3,2,figsize=(24, 16))

        fig.suptitle(cg, fontsize=16)

        sns.lineplot(data=temp, x=temp.index, y= y_true, ax=ax[0][0])

        sns.lineplot(data=temp, x=temp.index, y= y_predicted, ax=ax[0][0])

        sm.graphics.tsa.plot_acf(temp['residual'], lags = num_lags, alpha = 0.05, ax=ax[0][1])

        sm.graphics.tsa.plot_pacf(temp['residual'], lags =num_lags,alpha = 0.05, ax=ax[1][1])

        sns.lineplot(data=temp, x=temp.index, y='residual', ax=ax[1][0])

        sns.histplot(data=temp, x='residual',  ax=ax[2][0])

        alpha = 0.01
        
        #stat, p = shapiro(temp['Residuo'])

        texto0 = f"{temp.shape[0]} dias"
        media = fr'média = {round(temp.residual.mean(),3)}'
        mediana = fr'mediana = {round(temp.residual.median(),3)}'
        desvio = fr'desvio padrão = {round(temp.residual.std(),3)}'
        #normal = "Normal:" + ("Sim" if (p > alpha) else "Não")

        ax[2][1].text(0.5,0.8, media, horizontalalignment='center',fontsize = 25,
                      verticalalignment='center', transform = ax[2][1].transAxes)

        ax[2][1].text(0.5,0.6, mediana, horizontalalignment='center',fontsize = 25,
                      verticalalignment='center', transform = ax[2][1].transAxes)

        ax[2][1].text(0.5,0.4, desvio, horizontalalignment='center',fontsize = 25,
                      verticalalignment='center', transform = ax[2][1].transAxes)  

        ax[2][1].grid(False)

        plt.show()